In [97]:
%load_ext autoreload
%autoreload 2

import torch
import tqdm
import gpytorch


import numpy as np
from ase import io
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys
sys.path.append("../../fande") # Adds higher directory to python modules path.
sys.path.append("..") # Adds higher directory to python modules path.

In [100]:
path = os.getcwd()

print(path)

/home/dlb/coding/shared_coding/repos/chem-gp/saddle-dynamics/notebooks


In [545]:
from fande.data import FandeDataModule

### Parsing and loading data:

In [15]:
%load_ext autoreload
%autoreload 2

from sdynamics.load import parse_trajectories, parse_forces, flatten_trj_dictionaries

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [625]:
trajectories, energies_trj, trj_files_basenames = parse_trajectories(traj_folder="../data/dynamics/ene_grad_fitting/data/trj/")
forces_trj = parse_forces(forces_path = '../data/dynamics/ene_grad_fitting/data/grad/', trj_files_basenames = trj_files_basenames)

Reading trajectory files...


100%|██████████| 50/50 [00:18<00:00,  2.77it/s]

Trajectory files reading done!
Reading .npy files with forces...
Reading .npy files with forces done!


In [626]:
traj, energies, forces = flatten_trj_dictionaries(trajectories, energies_trj, trj_files_basenames, forces_trj)

Flattening done!


### Prepare training/test datasets:

In [949]:
# Training data:
# training_indices = np.sort(  np.random.choice(np.arange(0,10_000), 10000, replace=False) )
training_indices = np.sort(  np.arange(0, 90_000, 500) )  
traj_train = [traj[i] for i in training_indices]
energies_train = energies[training_indices]
forces_train = forces[training_indices]
train_data = {'trajectory': traj_train, 'energies': energies_train, 'forces': forces_train}

#Test data:
# test_indices = np.sort(  np.random.choice(np.arange(0,92795), 200, replace=False) ) 
test_indices = np.sort(  np.arange(0,1_000,1) ) 
traj_test = [traj[i] for i in test_indices]
energies_test = energies[test_indices]
forces_test = forces[test_indices]
test_data = {'trajectory': traj_test, 'energies': energies_test, 'forces': forces_test}

In [950]:
from fande.data import FandeDataModuleASE


hparams = {
    'dtype' : 'float64',
    'device' : 'gpu'
}

fdm = FandeDataModuleASE(train_data, test_data, hparams, atoms_forces=None)

In [951]:
soap_params = {
    'species': ["H", "C"],
    'periodic': False,
    'rcut': 3.0,
    'sigma': 0.5,
    'nmax': 5,
    'lmax': 5,
    'average': "outer",
    'crossover': True,
    'dtype': "float64",
    'n_jobs': 10,
    'sparse': False,
    'positions': [0,2,3,6]
}

fdm.calculate_invariants(soap_params)

Total length of train traj is 180
Starting SOAP calculation...
SOAP calculation done!
Total length of test traj is 1000
Starting SOAP calculation...
SOAP calculation done!
(180, 17, 3, 330)
(180, 330)


In [1025]:
fdm.train_DX.shape

torch.Size([9180, 330])

### Fitting Forces:

In [1024]:
from fande.models import ModelForces, ModelEnergies, MyCallbacks
from torch.utils.data import DataLoader, TensorDataset, random_split
from pytorch_lightning import Trainer, seed_everything
import numpy as np
# seed_everything(42, workers=True)

train_DX = fdm.train_DX
train_F = fdm.train_F
test_DX = fdm.test_DX
test_F = fdm.test_F

# ind_slice = np.sort( np.concatenate( 
#     ( np.arange(0,4800), np.arange(11*4800,12*4800), np.random.choice(np.arange(4800,59200), 300, replace=False) ) 
#     ) )

# ind_slice = np.sort(  np.random.choice(np.arange(0,train_F.shape[0]), 2_000, replace=False) ) 
# ind_slice = np.sort(  np.arange(0,2550) ) 
ind_slice = np.sort(  np.arange(0,train_F.shape[0]) ) 


train_dataset = TensorDataset(train_DX[ind_slice], train_F[ind_slice])
train_loader = DataLoader(train_dataset, batch_size=100_000)

model_f = ModelForces(train_DX[ind_slice], train_F[ind_slice], hparams, 0.05)

trainer_f = Trainer(
    gpus=0, 
    max_epochs=10, 
    precision=64,
    weights_summary='full', 
    callbacks=[MyCallbacks()])

trainer_f.fit(model_f, train_loader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/dlb/anaconda3/envs/pyc/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1566: UserWarning:

GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.


   | Name                                                      | Type                       | Params
----------------------------------------------------------------------------------------------------------
0  | likelihood                                                | GaussianLikelihood         | 1     
1  | likelihood.noise_covar                                    | HomoskedasticNoise         | 1     
2  | likelihood.noise_covar.raw_noise_constraint               | GreaterThan                | 0     
3  | likelihood.noise_covar.raw_noise_constraint._transform    | Softplus                   | 0     
4  | model                                                  


 setup() callback called...
Epoch 9: 100%|██████████| 1/1 [00:02<00:00,  2.01s/it, loss=0.708, v_num=183]

 teardown() callback called...


### Predictions on test data:

In [1026]:
from fande.predict import PredictorASE

test_X = fdm.test_X
test_DX = fdm.test_DX
test_E = fdm.test_E
test_F = fdm.test_F

model_e = None
trainer_e = None


predictor = PredictorASE(
            model_e,
            model_f,
            trainer_e,
            trainer_f,
            test_X,
            test_DX,
            test_E,
            test_F,
            test_data,
            hparams,
            soap_params
)


# predictor.predict_and_plot_forces()
mol = fdm.traj_test[0].copy()
# f_, f_var_ = predictor.predict_forces_single(mol)

print(f_.shape, f_var_)

(17, 3) [[0.03557588 0.02658165 0.02494853]
 [0.08654764 0.01939834 0.0330811 ]
 [0.04102013 0.02392031 0.02153549]
 [0.06177905 0.02531317 0.02900263]
 [0.01348011 0.01000791 0.01814585]
 [0.01911702 0.01420778 0.01322842]
 [0.01224534 0.01147615 0.00967989]
 [0.01266411 0.00939868 0.01064205]
 [0.01436309 0.00920342 0.01047329]
 [0.01263888 0.01136476 0.01360414]
 [0.01316529 0.01234817 0.01075965]
 [0.01782208 0.01375587 0.01647894]
 [0.01459548 0.01162721 0.02730875]
 [0.01009001 0.01038885 0.0120274 ]
 [0.01072296 0.01038919 0.00978262]
 [0.0073473  0.00807979 0.00805806]
 [0.0085479  0.00731232 0.00852278]]


In [1029]:
x,dx = predictor.soap_single(mol)

dx = dx.cpu()
# print(dx.device)
# mf_cpu = model_f.cpu()
# mf_cpu.model(dx)
model_f.model(dx)

Starting SOAP calculation...
SOAP calculation done!


RuntimeError: You must train on the training inputs!

In [1022]:
model_f.cpu()

ModelForces(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): ExactGPModelForces(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (covar_module): ScaleKernel(
      (base_kernel): RBFKernel(
        (raw_lengthscale_constraint): Positive()
        (distance_module): Distance()
      )
      (raw_outputscale_constraint): Positive()
    )
    (mean_module): ConstantMean()
  )
  (mll): ExactMarginalLogLikelihood(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (model): ExactGPModelForces(
      (likelihood): GaussianLikelihood(
        (noise_covar): HomoskedasticNoise(
          (raw_noise_constraint): GreaterThan(1.000E-04)
        )
      )
      (covar_module): ScaleKernel(
        

### Dynamics simulation

In [ ]:
from sdynamics.dynamics import MDynamics

atoms = ... # initialize saddle structure

atoms.calc = gp_model.ase_calc() # get calc from the trained GP

md_runner = MDynamics(atoms) # instantiate class for running MD

md_runner.run() # run the MD

### Testing ASE calc

In [ ]:
%%time
from fande.ase import FandeCalc
from ase.build import molecule

from ase import io

logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

traj = fdm.mol_traj #io.read("data/dump/mol_trj.xyz", index=":")
atoms = traj[0]

# atoms = molecule("CH3CH2OCH3")
fande_calc = FandeCalc(hparams, model_e, trainer_e, model_f, trainer_f)

for a in traj: a.calc=fande_calc 

print(atoms.get_potential_energy() )
print( atoms.get_forces())


In [ ]:
### Molecular Dynamics with Fande calculator

In [ ]:
%%capture
from fande.ase import MDRunner

atoms = fdm.mol_traj[10].copy()
atoms.calc = FandeCalc(hparams, model_e, trainer_e, model_f, trainer_f)

mdr = MDRunner(atoms, "data/dump/ase/md_test.xyz", "data/dump/ase/md_log.log")
mdr.run()

### Visualize:

In [519]:
from ase.visualize import view

# trj = trajectories[trj_files_basename[2]]
view(traj[0:100])
# view(traj[0:100])

<Popen: returncode: None args: ['/home/dlb/anaconda3/envs/pyc/bin/python', '...>